In [1]:
%load_ext autoreload
%autoreload 3

In [2]:
from datetime import datetime, timedelta

import pendulum

In [3]:
import sys
sys.path.insert(0, '..')

In [4]:
from tourney.tournament import Game, Match, MatchSeries, Tournament
from tourney.scheduler import get_priority_schedule, get_ref_schedule, get_scoreboard_schedule

In [5]:
teams = list('ABCDEF')
matches = MatchSeries.all_pairs(teams)
matches

A vs. B
A vs. C
A vs. D
A vs. E
A vs. F
B vs. C
B vs. D
B vs. E
B vs. F
C vs. D
C vs. E
C vs. F
D vs. E
D vs. F
E vs. F

In [6]:
start = pendulum.parse('2023-01-28 09:30:00-05:00')
dt = timedelta(minutes=25)
times = [start + i * dt for i in range(len(matches))]

In [7]:
tourney = Tournament.create(matches, times)
tourney

Tournament(15 games, 6 teams: ['A', 'B', 'C', 'D', 'E', 'F'], dates: ['2023-01-28'])

In [8]:
tourney.to_dataframe()

,game,time,match
0,1,2023-01-28 09:30:00-05:00,"(A, B)"
1,2,2023-01-28 09:55:00-05:00,"(A, C)"
2,3,2023-01-28 10:20:00-05:00,"(A, D)"
3,4,2023-01-28 10:45:00-05:00,"(A, E)"
4,5,2023-01-28 11:10:00-05:00,"(A, F)"
5,6,2023-01-28 11:35:00-05:00,"(B, C)"
6,7,2023-01-28 12:00:00-05:00,"(B, D)"
7,8,2023-01-28 12:25:00-05:00,"(B, E)"
8,9,2023-01-28 12:50:00-05:00,"(B, F)"
9,10,2023-01-28 13:15:00-05:00,"(C, D)"


In [14]:
scheduled_tourney = Tournament.create(get_priority_schedule(teams, seed=0), times)
scheduled_tourney.to_dataframe()

,game,time,match
0,1,2023-01-28 09:30:00-05:00,"(A, C)"
1,2,2023-01-28 09:55:00-05:00,"(D, F)"
2,3,2023-01-28 10:20:00-05:00,"(B, E)"
3,4,2023-01-28 10:45:00-05:00,"(C, F)"
4,5,2023-01-28 11:10:00-05:00,"(A, D)"
5,6,2023-01-28 11:35:00-05:00,"(B, F)"
6,7,2023-01-28 12:00:00-05:00,"(C, E)"
7,8,2023-01-28 12:25:00-05:00,"(A, F)"
8,9,2023-01-28 12:50:00-05:00,"(B, D)"
9,10,2023-01-28 13:15:00-05:00,"(A, E)"


In [25]:
refs, ref_counts = get_ref_schedule(teams, scheduled_tourney.matches)
for match, ref in zip(scheduled_tourney.matches, refs):
    print(match, '--', ref)
ref_counts

A vs. C -- D
D vs. F -- A
B vs. E -- F
C vs. F -- B
A vs. D -- C
B vs. F -- E
C vs. E -- D
A vs. F -- C
B vs. D -- A
A vs. E -- F
C vs. D -- B
E vs. F -- D
A vs. B -- E
D vs. E -- A
B vs. C -- F


{'D': 3, 'A': 3, 'F': 3, 'C': 2, 'B': 2, 'E': 2}

In [26]:
scores, score_counts = get_scoreboard_schedule(teams, scheduled_tourney.matches, refs)
for match, ref, score in zip(scheduled_tourney.matches, refs, scores):
    print(match, '--', ref, '--', score)
score_counts

A vs. C -- D -- E
D vs. F -- A -- C
B vs. E -- F -- A
C vs. F -- B -- E
A vs. D -- C -- B
B vs. F -- E -- C
C vs. E -- D -- F
A vs. F -- C -- B
B vs. D -- A -- F
A vs. E -- F -- D
C vs. D -- B -- E
E vs. F -- D -- A
A vs. B -- E -- C
D vs. E -- A -- B
B vs. C -- F -- D


{'F': 2, 'E': 3, 'A': 2, 'C': 3, 'B': 3, 'D': 2}